In [8]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler

# Carregar os dados
df = pd.read_csv('Reddit_Data.csv')
df_filtrado = df[df['category'] != 0]
df_1 = df_filtrado['clean_comment']
rotulos = df_filtrado['category']

df['Comentario'] = df['Comentario'].apply(lambda x: x.lower())

# Tokenização e lematização
comentarios_tokenizados = []
wnl = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    return [wnl.lemmatize(plv) for plv in tokens]

for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)

lemm_tks = [lemmatize_tokens(tokens) for tokens in comentarios_tokenizados]

# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(lemm_tks, rotulos, test_size=0.2, random_state=42)

# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])

# Aplicando Random Undersampling
undersampler = RandomUnderSampler()
X_treino_res, y_treino_res = undersampler.fit_resample(X_treino_vect, y_treino)

# Treinamento do classificador de Regressão Logística
classifier = LogisticRegression(max_iter=1000)

# Validação cruzada
scores = cross_val_score(classifier, X_treino_res, y_treino_res, cv=5)  # cv=5 indica 5-fold cross-validation

# Treinamento final do modelo
classifier.fit(X_treino_res, y_treino_res)

# Predição nos dados de teste
predicoes = classifier.predict(X_teste_vect)

# Calcular precisão, recall e F-score
precision = precision_score(y_teste, predicoes)
recall = recall_score(y_teste, predicoes)
f_score = f1_score(y_teste, predicoes)

# Imprimir resultados
print("Resultados da validação cruzada:")
print("Scores:", scores)
print("Precisão média:", scores.mean())
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))


Resultados da validação cruzada:
Scores: [0.87992424 0.88253126 0.87002653 0.87381584 0.86964759]
Precisão média: 0.8751890925166788
Precisão: 0.94
Recall: 0.87
F-score: 0.90


In [3]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
import re
from unidecode import unidecode

# Carregar os dados
# df = pd.read_csv('Reddit_Data.csv')
# df_filtrado = df[df['category'] != 0]
# df_1 = df_filtrado['clean_comment']
# rotulos = df_filtrado['category']

data = pd.read_csv('new_comentarios_ingles.csv')


# Removendo pontuação e emoticons
def remover_pontuacao(texto):
    new_teste = re.sub(r'["]', "", texto)
    return unidecode(re.sub(r"[,.\/'!@#\$\%\^\&\*\(\)\-\+\=\[\]\{\}\|\;\:\,\<\>\?´^~]", '', new_teste.lower()))

def remover_emoticons(texto):
    emoticon_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Símbolos e pictogramas
        "\U0001F680-\U0001F6FF"  # Transporte e símbolos de mapa
        "\U0001F1E0-\U0001F1FF"  # Bandeiras
        "\U00002700-\U000027BF"  # Diversos Símbolos e Pictogramas
        "\U00002600-\U000026FF"  # Diversos Símbolos
        "\U00002B50-\U00002B55"  # Símbolos e pictogramas adicionais
        "]+", flags=re.UNICODE)
    return emoticon_pattern.sub(r'', texto)


data['Comentario'] = data['Comentario'].apply(remover_pontuacao)
data['Comentario'] = data['Comentario'].apply(remover_emoticons)

df_1 = data['Comentario']
# rotulos = data['sentimento']

def define_sentiment(rating):
    if rating == 'positivo':
        return 1
    elif rating == 'neutro':
        return 2
    else:
        return 0
data['sentimento_polarizado'] = data['sentimento'].apply(define_sentiment)
rotulos = data['sentimento_polarizado']

# Tokenização e lematização
comentarios_tokenizados = []


for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)


# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(comentarios_tokenizados, rotulos, test_size=0.2, random_state=42, stratify=rotulos)

# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])

# Aplicando Random Oversampler
oversampler = RandomOverSampler()
X_treino_res, y_treino_res = oversampler.fit_resample(X_treino_vect, y_treino)

# Treinamento do classificador de Regressão Logística
classifier = LogisticRegression(max_iter=1000)

# Validação cruzada
scores = cross_val_score(classifier, X_treino_res, y_treino_res, cv=5)  # cv=5 indica 5-fold cross-validation

# Treinamento final do modelo
classifier.fit(X_treino_res, y_treino_res)

# Predição nos dados de teste
predicoes = classifier.predict(X_teste_vect)

# Calcular precisão, recall e F-score
precision = precision_score(y_teste, predicoes, average='weighted')
recall = recall_score(y_teste, predicoes, average='weighted')
f_score = f1_score(y_teste, predicoes, average='weighted')
acuracia = accuracy_score(y_teste, predicoes)
class_report = classification_report(y_teste, predicoes, target_names=['Negativo', 'Positivo', 'Neutro'])

# Imprimir resultados
print("Resultados da validação cruzada:")
print("Scores:", scores)
print("Precisão média:", scores.mean())
print("Acuracia: {:.2f}".format(acuracia))
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))
print("\nClassification Report:")
print(class_report)

Resultados da validação cruzada:
Scores: [0.94288628 0.94083896 0.94185898 0.9370139  0.94440903]
Precisão média: 0.9414014309248373
Acuracia: 0.77
Precisão: 0.80
Recall: 0.77
F-score: 0.78

Classification Report:
              precision    recall  f1-score   support

    Negativo       0.24      0.28      0.26       220
    Positivo       0.90      0.85      0.88      3268
      Neutro       0.18      0.25      0.21       317

    accuracy                           0.77      3805
   macro avg       0.44      0.46      0.45      3805
weighted avg       0.80      0.77      0.78      3805

